Analyze sentiment in Amazon product reviews and predict whether a review is positive or negative.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from sklearn.metrics import classification_report



In [3]:
df = pd.read_csv('amazon_reviews.csv')

In [4]:
df.head(5)

Unnamed: 0  reviewerName  overall  \
0           0           NaN      4.0   
1           1          0mie      5.0   
2           2           1K3      4.0   
3           3           1m2      5.0   
4           4  2&amp;1/2Men      5.0   

                                          reviewText  reviewTime  day_diff  \
0                                         No issues.  2014-07-23       138   
1  Purchased this for my device, it worked as adv...  2013-10-25       409   
2  it works as expected. I should have sprung for...  2012-12-23       715   
3  This think has worked out great.Had a diff. br...  2013-11-21       382   
4  Bought it with Retail Packaging, arrived legit...  2013-07-13       513   

   helpful_yes  helpful_no  total_vote  score_pos_neg_diff  \
0            0           0           0                   0   
1            0           0           0                   0   
2            0           0           0                   0   
3            0           0           0                   0   
4            0           0           0                   0   

   score_average_rating  wilson_lower_bound  
0                   0.0                 0.0  
1                   0.0                 0.0  
2                   0.0                 0.0  
3                   0.0                 0.0  
4                   0.0                 0.0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4915 non-null   int64  
 1   reviewerName          4914 non-null   object 
 2   overall               4915 non-null   float64
 3   reviewText            4914 non-null   object 
 4   reviewTime            4915 non-null   object 
 5   day_diff              4915 non-null   int64  
 6   helpful_yes           4915 non-null   int64  
 7   helpful_no            4915 non-null   int64  
 8   total_vote            4915 non-null   int64  
 9   score_pos_neg_diff    4915 non-null   int64  
 10  score_average_rating  4915 non-null   float64
 11  wilson_lower_bound    4915 non-null   float64
dtypes: float64(3), int64(6), object(3)
memory usage: 460.9+ KB


In [6]:
df.isnull().sum()

Unnamed: 0              0
reviewerName            1
overall                 0
reviewText              1
reviewTime              0
day_diff                0
helpful_yes             0
helpful_no              0
total_vote              0
score_pos_neg_diff      0
score_average_rating    0
wilson_lower_bound      0
dtype: int64

In [7]:
df = df.dropna(subset=['reviewText'])

In [8]:
df = df[['reviewText', 'overall']]

In [9]:
def get_actual_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating <= 2:
        return 'Negative'
    else:
        return 'Neutral'

df['actual_sentiment'] = df['overall'].apply(get_actual_sentiment)


In [10]:
import re

def preprocess_text_regex(text):
    text = text.lower()  
    text = re.sub(r"[^a-z\s]", "", text)  
    tokens = text.split()
    stop_words = set(["is", "a", "for", "and", "the", "this"])  
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)


text = "This is an excellent way to preprocess text for sentiment analysis!"
cleaned_text = preprocess_text_regex(text)
print(cleaned_text)


an excellent way to preprocess text sentiment analysis


In [11]:
df['cleaned_review'] = df['reviewText'].apply(preprocess_text_regex)


In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [13]:
df['sentiment_score'] = df['cleaned_review'].apply(lambda x: sid.polarity_scores(x)['compound'])

In [14]:
def classify_sentiment(score):
    if score >= 0.3:
        return 'Positive'
    elif score <= -0.3:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['sentiment_score'].apply(classify_sentiment)


In [15]:
df.head(5)

reviewText  overall  \
0                                         No issues.      4.0   
1  Purchased this for my device, it worked as adv...      5.0   
2  it works as expected. I should have sprung for...      4.0   
3  This think has worked out great.Had a diff. br...      5.0   
4  Bought it with Retail Packaging, arrived legit...      5.0   

  actual_sentiment                                     cleaned_review  \
0         Positive                                          no issues   
1         Positive  purchased my device it worked as advertised yo...   
2         Positive  it works as expected i should have sprung high...   
3         Positive  think has worked out greathad diff bran gb car...   
4         Positive  bought it with retail packaging arrived legit ...   

   sentiment_score sentiment  
0          -0.2960   Neutral  
1          -0.2960   Neutral  
2          -0.3089  Negative  
3           0.6486  Positive  
4           0.7087  Positive

In [16]:
sns.countplot(x='sentiment', data=df)
plt.title('Sentiment Distribution')
plt.show()


In [17]:
print(df[df['sentiment'] == 'Positive'].head(3)['reviewText'])
print(df[df['sentiment'] == 'Negative'].head(3)['reviewText'])
print(df[df['sentiment'] == 'Neutral'].head(3)['reviewText'])


3    This think has worked out great.Had a diff. br...
4    Bought it with Retail Packaging, arrived legit...
6    I have it in my phone and it never skips a bea...
Name: reviewText, dtype: object
2     it works as expected. I should have sprung for...
19    This works with the NL1520.  No video stutteri...
32    I bought 2 of those SanDisk 32 GB microSD , us...
Name: reviewText, dtype: object
0                                           No issues.
1    Purchased this for my device, it worked as adv...
5    It's mini storage.  It doesn't do anything els...
Name: reviewText, dtype: object


In [18]:

print(classification_report(df['actual_sentiment'], df['sentiment'], target_names=['Negative', 'Neutral', 'Positive']))


              precision    recall  f1-score   support

    Negative       0.26      0.48      0.34       324
     Neutral       0.04      0.26      0.06       142
    Positive       0.95      0.70      0.81      4448

    accuracy                           0.68      4914
   macro avg       0.42      0.48      0.40      4914
weighted avg       0.88      0.68      0.76      4914



In [19]:
df.to_csv('amazon_reviews_with_sentiments.csv', index=False)
